In [1]:
! nvidia-smi

Wed Jun 16 14:00:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Tesla V1...  On   | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import os
import glob
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from loader import FeatureLoader
from model import AttentionMIL, Extractor

ModuleNotFoundError: No module named 'torch'

In [7]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
def extract_patches(image, patch_size, strides):

    x, y = image.shape[:-1]

    grid_x = list(range(0, x-patch_size, strides)) + [x-patch_size]
    grid_y = list(range(0, y-patch_size, strides)) + [y-patch_size]

    indices = np.meshgrid(grid_x, grid_y, indexing='ij')
    indices = np.transpose(indices, (1,2,0)).reshape(-1, 2)

    patches = np.array([image[x:x+patch_size, y:y+patch_size] for (x,y) in indices])

    return patches, indices

In [4]:
def evaluate(model, dataloader, device):
    
    y_true = []
    y_pred = []
    
    model.eval()

    with torch.no_grad():

        for data in dataloader:

            X = data['X'].to(device)
            Y = data['Y'].to(device)

            # Forward pass
            P, A = model(X)
            P = F.softmax(P, dim=-1)
                        
            y_true.append(Y.cpu().numpy())
            y_pred.extend(P.cpu().numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
        
    return y_true, y_pred

In [5]:
config = {
    "data_dir": '/mnt/scratch/crc/data/images',
    "kimianet": '/mnt/scratch/crc/models/kimianet.h5',
    "model_dir": '/mnt/scratch/crc/models/models_0512',
    "save_dir": '/mnt/scratch/crc/outputs/outputs_0512',
    "patch_size": 512,
    "fold": 0,
    "seed": 0,
}

if not os.path.exists(config["save_dir"]):
    os.makedirs(config["save_dir"])      

NameError: name 'os' is not defined

In [7]:
print(f'\nFold-{config["fold"]} ...\n', flush=True)

# Arange files and labels
files = sorted(glob.glob(f'{config["data_dir"]}/*.png'))
labels = np.array([int(f.split('/')[-1][5]) for f in files]) - 1

# K-Fold Split
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
_, valid_indices = list(skf.split(files, labels))[config["fold"]]
valid_samples = [{"X": files[i], "Y":labels[i]} for i in valid_indices]

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load extractor
extractor = Extractor(config["kimianet"], (config["patch_size"],config["patch_size"],3))
print(f'Extractor loaded', flush=True)

# Load mil model
model = AttentionMIL(feature_size=1024, classes=3).to(device)
checkpoint_path = f'{config["model_dir"]}/model_f{config["fold"]}.pt'
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])
print(f'MIL model loaded', flush=True)


Fold-0 ...

Extractor loaded
MIL model loaded


In [8]:
c = 0

sample = valid_samples[c]

In [9]:
# Read image and label

image = np.array(Image.open(sample["X"]))
label = sample["Y"]

# Extract patches

patches, indices = extract_patches(image, patch_size=config["patch_size"], strides=config["patch_size"]//2)
print(f'Patches extracted', flush=True)

# Extract features

patches = tf.keras.applications.densenet.preprocess_input(patches)
features = extractor.predict(patches, batch_size=8, verbose=1)
print(f'Features extracted', flush=True)

Patches extracted


NotFoundError:  No algorithm worked!
	 [[node model/conv1/conv/Conv2D (defined at <ipython-input-9-29f1c9d4921a>:14) ]] [Op:__inference_predict_function_9700]

Function call stack:
predict_function


In [ ]:
# MIL Classification

features = torch.tensor(features).float().to(device)

with torch.no_grad():
    prediction, attention = mil_model(features)  

prediction = torch.argmax(prediction).item()
attention = attention.cpu().numpy()[0]

features = None

print(f'\nY_TRUE: {CLASSES[int(label)]}', flush=True)
print(f'Y_PRED: {CLASSES[prediction]}', flush=True)